In [3]:
import pandas as pd

In [6]:
df_train = pd.read_csv("./test.csv").drop("Unnamed: 0", axis=1)
# df_train.MIS_Status = 1 - df_train.MIS_Status

In [7]:
def dollar_to_float(df):
    for c in df.columns:
        ls = list(df[c])
        if "$" not in str(ls[0]):
            continue
        ls = [float(s.replace(",","").replace("$","")) for s in ls]
        df[c] = ls
    return df

In [8]:
df_train = dollar_to_float(df_train)
df_train

,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,RevLineCr,LowDoc,DisbursementDate,Sector,ApprovalDate,ApprovalFY,City,State,BankState,DisbursementGross,GrAppv,SBA_Appv,UrbanRural
0,5,2,1.0,1,0,0,T,N,31-Jul-04,23,6-Aug-07,2007,SUNNYVALE,CA,CA,25000.0,25000.0,21250.0,1
1,235,13,1.0,9,14,77725,Y,N,11-Jan-95,44,8-Mar-04,2004,PITTSBURGH,PA,PA,15000.0,15000.0,7500.0,0
2,31,5,2.0,0,0,0,N,NaN,NaN,56,27-Feb-07,2007,LITTLE ROCK,AR,AR,28000.0,28000.0,23800.0,1
3,120,4,1.0,0,1,0,Y,N,30-Apr-99,62,19-Dec-97,1998,LITTLE ROCK,AR,AR,7500.0,7500.0,6375.0,2
4,63,13,1.0,0,8,1,N,N,31-Dec-05,42,10-Jul-09,2009,Louisville,KY,DE,91000.0,93000.0,93000.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42303,243,10,1.0,3,14,0,N,N,1-Dec-12,42,23-Apr-12,2012,FT. WRIGHT,KY,OH,390000.0,150000.0,127500.0,0
42304,178,0,2.0,0,0,1,N,N,30-Nov-03,0,27-Oct-06,2007,PHILADELPHIA,PA,RI,100000.0,100000.0,90000.0,0
42305,42,1,2.0,3,9,0,Y,N,28-Feb-09,33,21-Sep-89,1989,ELMHURST,IL,IL,17000.0,17000.0,13600.0,0
42306,76,15,1.0,0,0,0,N,N,31-Jan-08,0,3-Apr-06,2006,NASHVILLE,TN,TN,7500.0,7500.0,6375.0,0


In [9]:
DATE_FMAT = '%d-%b-%y'
import numpy as np
from datetime import datetime as dt
def split_date(df):
    DisbursementDate_ls = df["DisbursementDate"]
    DisbursementDate_year = []
    DisbursementDate_month = []
    DisbursementDate_day = []
    DisbursementDate_weekday = []
    ApprovalDate_ls = df["ApprovalDate"]
    ApprovalDate_year = []
    ApprovalDate_month = []
    ApprovalDate_day = []
    ApprovalDate_weekday = []
    duration = []
    for i in range(len(df)):
        if type(DisbursementDate_ls[i]) == float:
            DisbursementDate_year.append(np.nan)
            DisbursementDate_month.append(np.nan)
            DisbursementDate_day.append(np.nan)
            DisbursementDate_weekday.append(np.nan)
        else:
            dd = dt.strptime(DisbursementDate_ls[i], DATE_FMAT)
            DisbursementDate_year.append(dd.year)
            DisbursementDate_month.append(dd.month)
            DisbursementDate_day.append(dd.day)
            DisbursementDate_weekday.append(int(dd.weekday()))
        if type(ApprovalDate_ls[i]) == float:
            ApprovalDate_year.append(np.nan)
            ApprovalDate_month.append(np.nan)
            ApprovalDate_day.append(np.nan)
            ApprovalDate_weekday.append(np.nan)
        else:
            ad = dt.strptime(ApprovalDate_ls[i], DATE_FMAT)
            ApprovalDate_year.append(ad.year)
            ApprovalDate_month.append(ad.month)
            ApprovalDate_day.append(ad.day)
            ApprovalDate_weekday.append(int(ad.weekday()))
        if type(DisbursementDate_ls[i]) != float and type(ApprovalDate_ls[i]) != float:
            duration.append((dd-ad).days)
        else:
            duration.append(np.nan)
    df["DisbursementDate_year"] = DisbursementDate_year
    df["DisbursementDate_month"] = DisbursementDate_month
    df["DisbursementDate_day"] = DisbursementDate_day
    df["DisbursementDate_weekday"] = DisbursementDate_weekday
    df["ApprovalDate_year"] = ApprovalDate_year
    df["ApprovalDate_month"] = ApprovalDate_month
    df["ApprovalDate_day"] = ApprovalDate_day
    df["ApprovalDate_weekday"] = ApprovalDate_weekday
    df["duration"] = duration
    return df

In [10]:
df_train = split_date(df_train)
df_train = df_train.drop("DisbursementDate", axis=1)
df_train = df_train.drop("ApprovalDate", axis=1)

In [11]:
import sklearn.preprocessing as sp
def df_onehot_enc(df, col):
    enc = sp.OneHotEncoder( sparse=False )
    RevLineCr_onehot = enc.fit_transform(pd.DataFrame(df_train[col]))
    for i in range(RevLineCr_onehot.shape[1]):
        df[col+"_%d"%i] = RevLineCr_onehot[:,i]
    return df

In [12]:
df_onehot_enc(df_train, "RevLineCr")
df_train = df_train.drop("RevLineCr", axis=1)
df_onehot_enc(df_train, "LowDoc")
df_train = df_train.drop("LowDoc", axis=1)
df_onehot_enc(df_train, "City")
df_train = df_train.drop("City", axis=1)
df_onehot_enc(df_train, "State")
df_train = df_train.drop("State", axis=1)
df_onehot_enc(df_train, "BankState")
df_train = df_train.drop("BankState", axis=1)

/home/u00232/anaconda3/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/u00232/anaconda3/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/u00232/anaconda3/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/tmp/ipykernel_30148/2678834969.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times

In [13]:
list(df_train.columns)

['Term',
 'NoEmp',
 'NewExist',
 'CreateJob',
 'RetainedJob',
 'FranchiseCode',
 'Sector',
 'ApprovalFY',
 'DisbursementGross',
 'GrAppv',
 'SBA_Appv',
 'UrbanRural',
 'DisbursementDate_year',
 'DisbursementDate_month',
 'DisbursementDate_day',
 'DisbursementDate_weekday',
 'ApprovalDate_year',
 'ApprovalDate_month',
 'ApprovalDate_day',
 'ApprovalDate_weekday',
 'duration',
 'RevLineCr_0',
 'RevLineCr_1',
 'RevLineCr_2',
 'RevLineCr_3',
 'RevLineCr_4',
 'LowDoc_0',
 'LowDoc_1',
 'LowDoc_2',
 'LowDoc_3',
 'LowDoc_4',
 'LowDoc_5',
 'LowDoc_6',
 'City_0',
 'City_1',
 'City_2',
 'City_3',
 'City_4',
 'City_5',
 'City_6',
 'City_7',
 'City_8',
 'City_9',
 'City_10',
 'City_11',
 'City_12',
 'City_13',
 'City_14',
 'City_15',
 'City_16',
 'City_17',
 'City_18',
 'City_19',
 'City_20',
 'City_21',
 'City_22',
 'City_23',
 'City_24',
 'City_25',
 'City_26',
 'City_27',
 'City_28',
 'City_29',
 'City_30',
 'City_31',
 'City_32',
 'City_33',
 'City_34',
 'City_35',
 'City_36',
 'City_37',
 'Cit

In [14]:
df_train.to_csv("df_test_onehot_enc.csv", index=None)

In [15]:
df_train

,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,Sector,ApprovalFY,DisbursementGross,GrAppv,...,BankState_43,BankState_44,BankState_45,BankState_46,BankState_47,BankState_48,BankState_49,BankState_50,BankState_51,BankState_52
0,5,2,1.0,1,0,0,23,2007,25000.0,25000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,235,13,1.0,9,14,77725,44,2004,15000.0,15000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,31,5,2.0,0,0,0,56,2007,28000.0,28000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,120,4,1.0,0,1,0,62,1998,7500.0,7500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,63,13,1.0,0,8,1,42,2009,91000.0,93000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42303,243,10,1.0,3,14,0,42,2012,390000.0,150000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42304,178,0,2.0,0,0,1,0,2007,100000.0,100000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42305,42,1,2.0,3,9,0,33,1989,17000.0,17000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42306,76,15,1.0,0,0,0,0,2006,7500.0,7500.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
